In [1]:
# gitHub kdt-service/dacon_titanic_8 리포지토리의 브랜치 kjw_Titanic 에 Task_KJW 디렉토리에 저장했습니다.

# 저는 학습수준이 낮아서 예측의 정확도를 높이는 시도보다 기본 개념을 이해하고 내재화 시키는 것을 목표로 정했습니다.

# 모델링의 단계를 3가지 로 이해했습니다.

#   첫째    데이터 프레임 :  입수한 화일로 ' 데이터 프레임 ' 이라는 것을 만든다.
#   둘째    전처리 : ' 데이터 프레임 '의 열(컬럼_key) 과 행(로우_value)를 info()메서드 등을 이용하여 분석하여 구조를 파악하고, 패키지에서 제공하는 기타 
#                       여러가지 메서드나 어트리뷰트를 사용하여 예측의 정확도를 높여줄 수 있도록 처리
                    # 컬럼의 값에 결측치(NaN)가 있는 경우 처리하는 방법 : 제거(dropna()메소드..) 혹은 추정하여 채움(loc인덱서, concat()메소드, fillna()메소드, one-hot인코딩)
                    # 변수에 할당 : 결측치를 처리한 데이터프레임을 모델 학습에 사용할 수 있도록 변수(입력데이터프레임 x_train, 정답데이터프레임 y_train)에 할당. 또한
                    #              학습한 모델을 사용하여 예측할 입력 데이터프레임도 변수(x_test)에 할당
#   셋째    모델링 : 사이킷런 패키지를 사용하여 전처리된 데이터프레임을 변수로 사용하여 모델을 학습시킨후 그 모델을 이용하여 예측(프리딕션 predict()메소드)을 실행한다.
                    # 랜덤포리스트 패키지를 사용했고, 사용자가 선택할 수 있는 매개변수 즉 하이퍼파라미터는 100에서 500까지 설정치를 변경해 보았슴.
                    # score()메소드를 사용하여 예측의 정확도를 확인했슴

# 학습에서 누락한 부분 : 시각화, 표준화, 정규화, K-fold 검증 등


# 분석 라이브러리
import numpy as np 
import pandas as pd
import random as rnd

# 시각화 라이브러리
import seaborn as sns
import matplotlib.pyplot as plt

#ML
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# 훈련데이터 셋과 테스트 데이터셋을 불러옵니다.
train_df = pd.read_csv("C:\\Users\\kjy15\\Desktop_MLP\\pjt_Titanic_8\\Task_KJW\\TT_3_train.csv")
test_df = pd.read_csv("C:\\Users\\kjy15\\Desktop_MLP\\pjt_Titanic_8\\Task_KJW\\TT_3_test.csv")

train_df.info()
print('#'*100)
print('#'*100)
test_df.info()
# Age의 결측치 처리가 필요함을 알 수 있습니다.
# Cabin의 결측치가 많음 해당 결측치를 어떻게 처리해야 할지 고민해야함 결측치 처리가 완료된 Cabin컬럼을 이용하여 새로운 feature를 발견하는것이 중요합니다.
# Embarked의 결측치 또한 존재합니다. 이를 어떻게 처리하면 좋을지 고민해보아야 합니다.
# 아래 info() 메서드 결과를 보시면 Age열과 Cabin열에 '결측치(=NaN, Null)' 가 있습니다. 이를 처리하는 과정을 진행해야 합니다. ==> 결측치 처리

# Variable	Definition	                                    Key                                             참고
# survival	Survival	                                    0 = No, 1 = Yes                                 생존여부
# pclass	    Ticket class	                                1 = 1st, 2 = 2nd, 3 = 3rd                       티켓등급
# sex	        Sex	                                                                                            성별
# Age	        Age in years	                                                                                나이
# sibsp	    # of siblings / spouses aboard the Titanic	                                                    동승한 형제자매/배우자
# parch	    # of parents / children aboard the Titanic	                                                    동승한 자녀/부모
# ticket	    Ticket number	                                                                                티켓번호    
# fare	    Passenger fare	                                                                                요금
# cabin	    Cabin number	                                                                                객실번호
# embarked	Port of Embarkation	                            C = Cherbourg, Q = Queenstown, S = Southampton  승선항구(셔버그/퀸즈타운/사우스햄튼)

print(train_df.columns)
test_df.columns
# 세부 컬럼은 아래와 같습니다.

train_df = train_df.drop(['Name'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
# pandas 라이브러리를 사용하여 데이터프레임에서 'Name' 열(column)을 
# 삭제하는 작업을 수행하는 코드입니다.
# train_df 와 test_df 두 개 데이터프레임에서, drop() 메서드를 사용하여 
# 'Name' 열을 삭제합니다. axis 매개변수를 1로 설정하여, 
# 열(column)을 삭제하도록 합니다. 만약, axis=0이라고 입력하면,
# 열(column)이 아닌 행(row)을 삭제하게 됩니다. 이 데이터프레임에서
# 'Name'이라는 행(row)가 없기 때문에 아무런 변화가 없을 것입니다.
# 즉, axis=1 과 axis=0 은 각각 열(column)과 행(row)을 의미하므로, 
# 해당 코드에서는 axis=1을 사용해야 'Name' 열을 삭제할 수 있습니다.
# 즉, 'Name' 열이 없는 새로운 데이터프레임을 
# train_df 와 test_df 로 업데이트합니다.
# 이 코드는 머신러닝 모델에서 불필요한 열을 제거하는 
# 전처리(preprocessing) 단계에서 사용합니다. 
# 이 경우 'Name' 열이 모델에 필요하지 않은 정보일 때, 
# 해당 열을 삭제하여 모델의 정확도를 높일 수 있습니다.
# 한가지 주의하실 사항은 TT_3_train.csv나 TT_3_test.csv를 불러와서 
# 만든 데이터프레임에서 삭제를 한 것이기 때문에 
# TT_3_train.csv나 TT_3_test.csv화일의 'Name'컬럼은 삭제되지 않는다는 것입니다.

#train_df.head()  # 5개 행(0행부터 4행까지) 보여줍니다.

train_df.pivot_table(values = 'Age', index = 'Pclass', aggfunc = 'mean')
# Pclass등급에 따른 나이 평균을 정리해보면 다음과 같다. 이를 이용해 결측값을 채워보려고 합니다.
# pandas 라이브러리를 사용하여 train_df 데이터프레임에서 'Pclass' 열(column)을 
# 기준으로 그룹화하여 'Age' 열의 평균(mean) 값을 구하는 작업을 수행하는 코드입니다.
# pivot_table() 메서드를 사용하여 데이터프레임을 피봇 테이블(pivot table) 형태로 변환합니다.
# values 매개변수에 'Age'를 입력하여, 'Age' 열의 값들을 테이블의 값(values)으로 사용합니다.
# index 매개변수에 'Pclass'를 입력하여, 'Pclass' 열의 고유값들을 기준(index)으로 행(row)을 그룹화합니다.
# aggfunc 매개변수에 'mean'을 입력하여, 각 그룹에서 'Age' 열의 평균(mean) 값을 계산합니다.
# 결과적으로 'Pclass' 열의 고유값들을 기준으로 그룹화된 피봇 테이블이 반환되며, 
# 각 그룹의 'Age' 열의 평균값이 계산되어 포함됩니다. 
# 이를 통해 클래스(Pclass)가 높을 수록 연령(Age)이 많은지, 적은지 등을 알아볼 수 있습니다.

train_df.pivot_table(aggfunc = 'mean', values = 'Age', index = 'Pclass')  
# .pivot_table()메서드 안의 인자들 순서를 바꿔서 써도 동일한 결과다.

train_age_null = train_df[train_df.Age.isnull()]
# ' Age '컬럼(열)에서 null값을 가지는 행 즉 결측값만을 가지는 
# ' 새로운 데이터 프레임 '을 생성하여, 그 이름을 train_age_null로 하는 코딩입니다.
# train_df 데이터프레임에서 'Age' 열(column) 값이 
# null 인 행(row)을 추출하는 작업을 수행하는 코드입니다.
# train_df.Age.isnull() 코드는 'Age' 열의 값이 null 인지 
# 아닌지를 검사하여, boolean(True/False) 형태의 
# 시리즈(series)를 반환합니다. 이 시리즈를 train_df 데이터프레임의 
# 인덱싱(indexing)에 활용하여, 'Age' 열의 값이 null 인 행(row)만을 
# 선택하도록 합니다. 이를 train_age_null 변수에 할당합니다. 변수에 넣어주는 것입니다.
# 따라서, train_age_null 변수에는 'Age' 열의 값이 null 인 행(row)들만을 포함하는 
# 새로운 ' 데이터프레임 '이 저장됩니다. 이 ' 데이터프레임 '을 활용하여 
# null 값을 처리하거나 다른 분석 작업을 수행할 수 있습니다.

train_age_null.info()  
# 아래 info() 메서드 결과는 Age열 값이 Null 인 177개 행에 대한 것이다. Age열은 Non_Null Count가 0 이다.

train_age_null.head(2)  # Age 컬럼의 값들이 NaN 인 것을 볼 수 있다.

train_first = train_age_null[train_age_null.Pclass == 1]
train_second = train_age_null[train_age_null.Pclass == 2]
train_third = train_age_null[train_age_null.Pclass == 3]

# ' train_age_null 데이터프레임 '에서 'Pclass' 열(column) 값이 각각 
#  1, 2, 3인 행(row)들을 추출하는 작업을 수행하는 코드입니다.
# train_age_null[train_age_null.Pclass == 1] 코드는 
# 'Pclass' 열의 값이 1(1등티켓을 의미)인 행(row)들을 선택하는 작업을 수행합니다. 
# 이를 train_first 변수에 할당합니다.
# train_age_null[train_age_null.Pclass == 2] 코드는 'Pclass' 열의 값이 2(2등티켓을 의미)인 
# 행(row)들을 선택하는 작업을 수행합니다. 이를 train_second 변수에 할당합니다.
# train_age_null[train_age_null.Pclass == 3] 코드는 'Pclass' 열의 값이 3인 행(row)들을 
# 선택하는 작업을 수행합니다. 이를 train_third 변수에 할당합니다.
# 따라서, train_first, train_second, train_third 변수에는 
# train_age_null 데이터프레임에서 'Age' 열의 값이 null 이며, 
# 'Pclass' 열의 값이 각각 1, 2, 3인 행(row)들만을 포함하는 
# 새로운 데이터프레임이 저장됩니다. 
# 이를 활용하여 각 클래스(Pclass)별로 null 값을 처리하거나 다른 분석 작업을 수행할 수 있습니다.

train_first.info()
print('\n')
train_second.info()
print('\n')
train_third.info()

train_firstclass = train_first[train_first.Age.isnull()].fillna(value = '38')
train_secondclass = train_second[train_second.Age.isnull()].fillna(value = '30')
train_thirdclass = train_third[train_third.Age.isnull()].fillna(value = '25')
# 3개의 각 데이터프레임(train_firstclass, train_secondclass, train_thirdclass)에서 Age 컬럼(열)에만 존재하는 
# 결측값(null)을 fillna()메서드를 사용하여 
# 이미 계산해 놓은 Pclass등급에 따른 나이 평균으로 채워주는 코딩입니다.

train_df[train_df.Age.isnull()]  
# train_df 데이터 프레임에서 'Age' 열(컬럼)의 값이 null(=NaN)인 행을 모두 보여주고 있습니다.
# 아래 결과값을 보면, 891개 행중에서 Age 컬럼의 값이 null(NaN)인 177개 행을 보여주고 있습니다.

train_drop_na = train_df.dropna(subset = ['Age'])
# ' train_df 데이터프레임 '에서 'Age' 열(column) 값이 null 인 경우 
# 해당 행(row)을 제거(drop)하여 ' train_drap_na라는 데이터프레임 '을 
# 만드는 작업을 수행하는 코드입니다.
# train_df.dropna(subset = ['Age']) 코드는 train_df 데이터프레임에서 
# 'Age' 열의 값이 null 인 행(row)을 제거합니다. 이를 ' train_drop_na 변수 '에 할당합니다.
# 따라서, train_drop_na 변수에는 train_df 데이터프레임에서 'Age' 열의 값이 
# null 이 아닌 행들만을 포함하는 ' 새로운 데이터프레임 '이 저장됩니다.
# subset = ['Age']는 dropna() 메서드에 전달되는 인자(argument)로, 
# 제거하고자 하는 null 값을 포함하는 열(column)을 선택하는 역할입니다.
# 즉, 해당 코드에서는 train_df 데이터프레임에서 'Age' 열(column)에 null 값이 
# 포함된 행(row)을 제거하고자 하기 때문에, subset 인자로 'Age' 열을 선택하였습니다. 
# 만약 다른 열에 null 값이 포함되어 있는 경우 이를 제거하고자 할 경우, subset 인자에 
# 해당 열의 이름을 전달하면 됩니다.
# subset 인자를 생략하면 디폴트로 모든 열(column)에 대해 null 값을 확인하고 제거합니다.

train_drop_na.info()  
# 아래 결과값을 보면, 891개 행중에서 Age 컬럼의 값이 null(NaN)인 177개 행이 제거되고 714개행이 있다.

train_drop_na.Cabin.info()
print('\n ')
train_firstclass.Cabin.info()
print('\n ')
train_secondclass.Cabin.info()
print('\n ')
train_thirdclass.Cabin.info()
# 아래 info() 메서드 결과를 보시면, 4개의 데이터 프레임 의 Cabin 컬럼 Non-Null Count를 확인할 수 있습니다. 
# concat()메서드를 사용하면 Cabin컬럼의 행들중에서 중복되는 행들이 발생됨을
# 알 수 있습니다. 

train_concat = pd.concat([train_drop_na, train_firstclass, train_secondclass, train_thirdclass])
train_df = train_concat
train_df.info()
# 결측값을 제거한 테이블과 결측값을 포함하지 않는 테이블을 병합한다.
# 데이터프레임을 합치고, 다시 할당(assign)하는 작업을 수행했습니다.
# pd.concat([train_drop_na, train_firstclass, train_secondclass, train_thirdclass]) 
# 코드는 train_drop_na, train_firstclass, train_secondclass, train_thirdclass 
# 데이터프레임을 순서대로 합칩니다. 이를 train_concat 변수에 할당합니다.
# train_df = train_concat 코드는 합쳐진 train_concat 데이터프레임을 
# 다시 train_df 변수에 할당합니다.
# train_df.info() 코드는 train_df 데이터프레임의 정보를 출력합니다. 
# info() 메서드는 데이터프레임의 열(column) 정보, null 값의 유무, 데이터타입 등의 기본 정보를 제공합니다.
# 따라서, 해당 코드를 실행하면 train_drop_na, train_firstclass, train_secondclass, 
# train_thirdclass 데이터프레임이 순서대로 합쳐진 train_df 데이터프레임이 생성됩니다. 
# train_df.info() 메서드로 데이터프레임의 정보를 확인할 수 있습니다.
# 데이터를 살펴보면 결측값은 없어졌지만 Age컬럼의 데이터 타입이 
# 숫자만 입력했음에도 오브젝트로 나와있습니다. 이를 정수형태로 변환하는 작업이 필요합니다.
# ****** concat() 메서드 수행후 'Cabin' 컬럼(열)의 non-Null Count가 204개에서 362개로 늘어났는데, 그 이유를 찾고 있습니다. 도와주세요. 
# ****** 제가 찾은 답은 위 info() 메서드에서 보시듯이 4개의 데이터프레임(rain_drop_na, train_firstclass, train_secondclass, train_thirdclass)
# ****** Cabin 컬럼의 Non-Null Count의 합이 362개입니다. 이것은 concat() 메서드를 통해 병합하면서 4개 데이터프레임의 Cabin 컬럼은 중복되어 병합된 것임을 알 수 있습니다.
# ****** Cabin 컬럼을 예측의 중요한 변수로 사용할 예정이라면 이렇게 하면 안됩니다. Cabin 컬럼을 예측의 중요 변수로 사용하지 않겠다면 이렇게 해도 됩니다.

train_df

train_df = train_df.astype({'Age':'int'})
train_df.info()
# 데이터를 정수형태로 변환하고 이를 다시 train_df에 담습니다.
# train_df.info()를 사용해 확인해보면 다음과 같습니다.
# DataFrame의 astype() 함수를 이용하여 'Age' 컬럼의 데이터 타입을 int로 변환하는 작업을 수행합니다.
# astype() 함수는 DataFrame 또는 Series의 데이터 타입을 변경할 수 있는 함수입니다. 
# 인자로는 변경할 데이터 타입을 딕셔너리 형태로 전달합니다. 딕셔너리의 key는 변경하고자 하는 컬럼의 이름이며, 
# value는 변경하고자 하는 데이터 타입입니다. 위 코드에서는 {'Age':'int'}라는 딕셔너리를 전달하고 있습니다. 
# 따라서 'Age' 컬럼의 데이터 타입을 int로 변경하겠다는 의미입니다. 이때, 'Age' 컬럼의 데이터 타입이 
# 정수형인 경우에는 별도의 처리가 필요하지 않지만, 실수형이나 문자형과 같은 다른 데이터 타입인 경우에는 
# 정확한 변환이 이루어지지 않을 수 있습니다. 따라서 데이터를 분석할 때는 데이터의 형식을 확인하고, 
# 이를 적절하게 변환하는 작업이 필요합니다. 위 코드에서는 'train_df' DataFrame 객체의 'Age' 컬럼의 
# 데이터 타입을 int로 변경하고 있으므로, 'Age' 컬럼의 모든 데이터가 정수형으로 변경됩니다.

test_age_null = test_df[test_df.Age.isnull()]
# 결측값을 가지는 새로운 데이터 프레임을 생성한다.

# 각 데이터를 1,2,3등급에 나누어서 
test_firstclass = test_age_null[test_age_null.Pclass == 1]
test_second = test_age_null[test_age_null.Pclass == 2]
test_third = test_age_null[test_age_null.Pclass == 3]

test_firstclass = test_firstclass.fillna(value = '38')
test_secondclass = test_second.fillna(value = '30')
test_thirdclass = test_third.fillna(value = '25')

# Age의 결측값을 포함하는 행을 삭제한다.
test_drop_na = test_df.dropna(subset = ['Age'])

#결측값을 제거한 테이블과 결측값을 포함하지 않는 테이블을 병합한다.
test_concat = pd.concat([test_drop_na, test_firstclass, test_secondclass, test_thirdclass])
test_df = test_concat
# test_df.info()
# 데이터를 살펴보면 결측값은 없어졌지만 Age컬럼의 데이터 타입이 
#숫자만 입력했음에도 오브젝트로 나와있습니다. 이를 정수형태로 변환하는 작업이 필요합니다.

# 데이터를 정수형태로 변환하고 이를 다시 train_df에 담습니다.
# train_df.info()를 사용해 확인해보면 다음과 같습니다.
test_df = test_df.astype({'Age':'int'})
test_df.info()

train_df["Embarked"].fillna('S', inplace=True)
test_df["Embarked"].fillna('S', inplace=True)
train_df.info()
test_df.info()
test_df["Fare"].fillna(0, inplace=True)
# Embarked(승선지)의 결측값은 두가지 방식의 처리를 제안합니다.
#   1. 다수인S로 채우는 방법
#   2. 해당 로우를 제거하는 방법
# Embarked(승선지)의 결측값은 첫번째를 선택해 진행하였습니다.
# 테스트 데이터셋에 있는 Fare결측값을 '0'으로 채워주는 코딩입니다.
#  inplace=True 옵션을 사용하면 대체한 값이 데이터프레임에 직접
# 적용되므로 즉 데이터프레임 자체를 수정하므로, 
# 별도의 할당(assign) 작업이 필요없습니다.

# one-hot-encoding 사용

pclass_train_dummies = pd.get_dummies(train_df['Pclass'])
pclass_test_dummies = pd.get_dummies(test_df['Pclass'])

train_df.drop(['Pclass'], axis=1, inplace=True)
test_df.drop(['Pclass'], axis=1, inplace=True)

pclass_train_dummies.columns = ['First', 'Second', 'Third']
pclass_test_dummies.columns = ['First', 'Second', 'Third']

train_df = train_df.join(pclass_train_dummies)
test_df = test_df.join(pclass_test_dummies)

# train_df와 test_df 데이터프레임에서 'Pclass' 열(column)의 값을 
# one-hot 인코딩(one-hot encoding)하여, 새로운 열을 추가하는 작업을 수행합니다.
# one-hot 인코딩(one-hot encoding)은 범주형(categorical) 데이터를 처리하는 방법 중 하나입니다.
# 이 방법은 각 범주(category)를 이진(binary) 변수로 변환하여 데이터를 표현합니다. 
# 예를 들어, "색깔"이라는 범주형 변수가 있다면, "빨강", "파랑", "노랑" 등과 같은 
# 각각의 카테고리를 새로운 이진 변수로 나타냅니다. 이진 변수는 0과 1로 구성되어 있으며, 
# 해당 카테고리가 적용되는 경우 1이 되고, 그렇지 않으면 0이 됩니다. 
# 이렇게 하면 모든 카테고리를 포함하는 벡터를 만들 수 있습니다. 
# 이 방법을 사용하면 범주형 데이터를 수치 데이터로 변환하여 분석에 사용할 수 있습니다.
# 이진 벡터는 해당 범주에 속하는 위치에 1의 값을 가지고, 그 외에는 모두 0의 값을 가집니다.
# 예를 들어, 다음과 같은 세 가지 색깔이 있는 경우: 빨간색 파란색 녹색
# 원핫인코딩을 적용하면, 이를 다음과 같은 이진 벡터로 표현할 수 있습니다:
# 빨간색: [1, 0, 0]  파란색: [0, 1, 0]  녹색: [0, 0, 1]
# 자연어 처리에서 텍스트 데이터를 벡터화할 때 활용합니다.
# 예를 들어 위 코드에서 
# pclass_train_dummies = pd.get_dummies(train_df['Pclass']) 코드는 
# train_df 데이터프레임에서 'Pclass' 열(column)의 값을 one-hot 인코딩하여, 
# 새로운 데이터프레임 pclass_train_dummies에 저장합니다.
# 여기서 pd.get_dummies() 메서드는 pd.get_dummies() 메서드는 Pandas에서 제공하는 
# 함수 중 하나로, 범주형 변수를 One-Hot Encoding 형태로 변환해주는 메서드입니다.
# pd.get_dummies() 함수는 범주형 변수를 인코딩하기 위해 pandas.DataFrame 또는 
# pandas.Series 데이터를 입력으로 받아서, 입력 데이터 내의 모든 범주(category)를 
# 각각 새로운 이진 열(binary column)로 변환합니다. 
# 이진 열 중에서 해당 범주에 해당하는 열에는 1의 값을, 
# 그 외에는 0의 값을 할당합니다. 따라서, pd.get_dummies() 메서드는 
# 범주형 변수를 One-Hot Encoding 형태로 변환하여 반환합니다.
# 예를 들어, 아래와 같은 예제 데이터가 있다고 가정해보겠습니다.
#   import pandas as pd
#   df = pd.DataFrame({
#         'class': ['A', 'B', 'C', 'A', 'B', 'C'],
#         'score': [90, 85, 92, 89, 88, 95]
#   })
# 이 데이터에서 'class' 열은 범주형 변수이므로, pd.get_dummies() 메서드를 
# 사용하여 아래와 같이 One-Hot Encoding을 적용할 수 있습니다.
#   dummies = pd.get_dummies(df['class'])
# 이렇게 적용하면, 'class' 열이 A, B, C로 이루어져 있으므로 총 3개의 이진 열이 생성됩니다.
#   dummies
#      A  B  C
#   0  1  0  0
#   1  0  1  0
#   2  0  0  1
#   3  1  0  0
#   4  0  1  0
#   5  0  0  1
# 위에서 'score'는 범주형 데이터가 아닙니다. 수치형 데이터입니다.
# 범주형 데이터는 데이터의 종류가 명확하게 정해진(discrete) 데이터를 의미합니다. 
# 범주형 데이터는 데이터를 구성하는 값이 명확하게 구분될 수 있으며, 
# 이들 값은 상호 배타적인(disjoint) 관계를 가지며, 순서나 크기의 개념이 적용되지 않는 경우가 많습니다.
# 예를 들어, 성별은 범주형 데이터의 예입니다. 성별은 남성(male) 또는 여성(female) 두 가지 값으로 
# 구분될 수 있으며, 이들 값은 서로 배타적입니다. 즉, 어떤 개체가 남성인 경우 여성이 될 수 없고, 
# 그 반대도 마찬가지입니다. 또한, 성별에 대해 "남성이 더 크다" 또는 "여성이 더 작다"와 같은 크기 비교를 할 수 없습니다.
# 다른 예로는 학교 성적 등급(A, B, C, D, F)이나 인종(아시아인, 백인, 흑인, 기타) 등도 범주형 데이터의 예입니다. 
# 이러한 범주형 데이터는 기계학습 알고리즘에서 입력 변수로 사용될 때, 적절한 전처리를 거쳐 수치형 데이터로 변환해야 합니다. 
# 이를 위해 One-Hot Encoding 등의 방법이 주로 사용됩니다.
# 수치형 데이터는 숫자 형태로 나타낼 수 있는 데이터로, 대부분의 경우 연속형(continuous) 데이터로 취급됩니다. 
# 'score' 열에서는 90, 85, 92, 89, 88, 95와 같은 실제 숫자가 사용되었습니다. 
# 범주형 데이터와는 달리, 수치형 데이터는 연산이 가능하고 크기 비교도 가능합니다. 
# 따라서, 수치형 데이터는 주로 회귀(regression) 분석이나 분류(classification) 분석 등에서 사용
# train_df.drop(['Pclass'], axis=1, inplace=True) 코드는 train_df 데이터프레임에서 'Pclass' 열(column)을 제거합니다. 
# axis=1 옵션은 열(column)을 제거하라는 의미입니다. inplace=True 옵션을 사용하여, train_df 데이터프레임 자체를 수정합니다.
# pclass_train_dummies.columns = ['First', 'Second', 'Third'] 코드는 pclass_train_dummies 데이터프레임의 
# 열(column) 이름을 'First', 'Second', 'Third'으로 변경합니다.
# train_df = train_df.join(pclass_train_dummies) 코드는 train_df 데이터프레임과 pclass_train_dummies 데이터프레임을 
# 병합하여, train_df 데이터프레임에 'First', 'Second', 'Third' 열(column)을 추가합니다.
# 따라서, 위 코드를 실행하면 train_df와 test_df 데이터프레임에서 'Pclass' 열(column)의 값을 
# one-hot 인코딩하여, 새로운 열(column)을 추가합니다. 이후, 'Pclass' 열(column)을 제거하고, 
# one-hot 인코딩된 열(column)의 이름을 변경합니다. 

train_df.head()

test_df.head()

sex_train_dummies = pd.get_dummies(train_df['Sex'])
sex_test_dummies = pd.get_dummies(test_df['Sex'])

sex_train_dummies.head(2)

sex_test_dummies.head(2)

sex_train_dummies.columns = ['Female', 'Male']
sex_test_dummies.columns = ['Female', 'Male']

sex_train_dummies.head(3)

sex_test_dummies.head(2)

train_df.drop(['Sex'], axis=1, inplace=True)
test_df.drop(['Sex'], axis=1, inplace=True)

train_df = train_df.join(sex_train_dummies)
test_df = test_df.join(sex_test_dummies)

train_df.head(3)

test_df.head(2)

embarked_train_dummies = pd.get_dummies(train_df['Embarked'])
embarked_test_dummies = pd.get_dummies(test_df['Embarked'])

embarked_train_dummies.columns = ['S', 'C', 'Q']
embarked_test_dummies.columns = ['S', 'C', 'Q']

embarked_train_dummies.head(2)

train_df.drop(['Embarked'], axis=1, inplace=True)
test_df.drop(['Embarked'], axis=1, inplace=True)

train_df.head(3)

train_df = train_df.join(embarked_train_dummies)
test_df = test_df.join(embarked_test_dummies)

train_df.head(2)

test_df.head(2)

train_df.info()

#결측치가 많은 Cabin컬럼과 예측에 도움이 되지 않을 것이라고 판단되는 Ticket컬럼, PassengerID컬럼은 제거함
train_df = train_df.drop(['Cabin', 'Ticket','PassengerId'], axis=1)
test_df = test_df.drop(['Cabin', 'Ticket'], axis=1)

train_df.info()
print('-'*80)
test_df.info()
# 결과를 보면 결측치가 있는 컬럼이 없습니다.

train_df.head()

X_train = train_df.drop("Survived",axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId",axis=1).copy()

# X_train = train_df.drop("Survived", axis=1) 코드는
# train_df 데이터프레임에서 'Survived' 열을 제외한 나머지 열들을 X_train 변수에 할당합니다. 
# 'Survived' 열은 타깃 변수로 사용됩니다. 따라서, 이 열은 모델링에 사용하지 않고 ,
# 입력 변수로 사용될 열들만 X_train에 포함됩니다.
# Y_train = train_df["Survived"] 코드는
# train_df 데이터프레임에서 'Survived' 열만 Y_train 변수에 할당합니다. 
# 'Survived' 열은 타깃 변수로 사용됩니다.
# X_test = test_df.drop("PassengerId", axis=1).copy() 코드는
# test_df 데이터프레임에서 'PassengerId' 열을 제외한 나머지 열들을 X_test 변수에 할당합니다. 
# 'PassengerId' 열은 예측 결과를 제출하기 위한 식별자로 사용됩니다. 따라서, 이 열은 모델링에 사용하지 않고,
# 입력 변수로 사용될 열들만 X_test에 포함됩니다. copy() 메서드를 사용하여 새로운 데이터프레임 객체를 생성합니다.
# 즉, 위 코드는 train_df와 test_df 데이터프레임을 분리하여 입력 변수와 타깃 변수를 각각 할당하는 과정입니다. 
# 이후에는 X_train, Y_train, X_test를 이용하여 모델을 학습하고 예측을 수행합니다.

X_train.head()

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)

# Python의 scikit-learn 라이브러리를 사용하여 앙상블(Ensemble) 학습 기법의 하나인 랜덤 포레스트 모델로 학습하고, 
# 테스트 데이터에 대한 예측을 수행하는 과정입니다.
# 첫째 줄: n_estimators=100으로 설정하여 랜덤 포레스트 모델을 생성합니다. n_estimators는 생성할 트리의 개수를 의미합니다.
#       참고로,    n_estimators는 랜덤 포레스트(Random Forest) 모델에서 생성할 의사결정나무(Decision Tree)의 개수를 
#       지정하는 하이퍼파라미터(Hyperparameter)입니다. 의사결정나무는 랜덤하게 추출된 샘플 데이터와 변수(Feature)를 사용하여 
#       학습하므로, 랜덤 포레스트에서 생성된 의사결정나무의 개수가 많을수록 모델의 정확도가 높아질 수 있습니다.
#       하지만,,, 의사결정나무의 개수가 너무 많으면 모델의 학습 시간이 늘어날 뿐만 아니라, 
#       모델의 복잡도가 증가하여 과적합(Overfitting) 문제가 발생할 수 있습니다. 
#       따라서 n_estimators는 모델의 성능과 학습 시간, 과적합 문제를 고려하여 적절한 값을 선택해야 합니다.
#       일반적으로 n_estimators 값은 100 ~ 500 사이의 값으로 설정하며, 데이터의 양과 특성, 컴퓨팅 자원 등에 따라 
#       조정할 필요가 있습니다. 또한, n_estimators 값이 작을수록 모델이 빠르게 학습되고 예측할 수 있습니다. 
#       하지만 일정 수준 이상의 값을 설정하지 않으면 모델의 성능이 좋아지지 않는 경우도 있습니다.
#       하이퍼파라미터(Hyperparameter)는 머신러닝 알고리즘을 구성하는 과정에서 사용자가 직접 설정해야 하는 파라미터입니다. 
#       즉, 모델 학습 시 데이터로부터 스스로 학습되는 파라미터와는 달리, 사용자가 직접 설정하는 파라미터입니다.
#       하이퍼파라미터는 모델의 구조와 학습 과정에 영향을 주며, 적절한 값을 설정하지 않으면 모델의 성능이 좋지 않을 수 있습니다. 
#       예를 들어, 의사결정나무 모델에서는 트리의 깊이나 가지치기(pruning) 기법에 대한 하이퍼파라미터를 설정할 수 있습니다. 
#       또한, 랜덤 포레스트(Random Forest) 모델에서는 의사결정나무의 개수나 랜덤 샘플링을 위한 샘플 크기 등에 대한 하이퍼파라미터를 설정할 수 있습니다.
#       하이퍼파라미터는 경험적으로 최적의 값을 찾거나, 그리드 탐색(Grid Search) 등의 최적화 알고리즘을 사용하여 자동으로 탐색할 수 있습니다. 
#       하지만 하이퍼파라미터를 탐색하면서 모델을 다시 학습해야 하므로, 이 작업은 시간과 자원이 많이 소모되는 작업 중 하나입니다.
# 둘째 줄: X_train, Y_train 데이터를 이용하여 랜덤 포레스트 모델을 학습합니다.
#       참고로, fit() 메서드는 머신러닝 알고리즘에서 사용되는 핵심 메서드 중 하나로, 모델을 학습하는 역할을 합니다. 즉, 
#       주어진 데이터를 모델에 적용하여 최적의 파라미터를 학습하고, 학습된 모델을 반환합니다.
#       Random Forest 모델에서 fit() 메서드는 입력 데이터인 X_train과 그에 해당하는 정답 데이터인 Y_train을 사용하여 
#       모델을 학습시키는 역할을 합니다. 이 때, 모델이 학습하는 과정은 다수의 의사결정나무(Decision Tree)를 생성하고 
#       이들을 앙상블하는 과정을 거칩니다. 이렇게 생성된 다수의 의사결정나무가 각각의 데이터를 예측한 결과를 종합하여 최종 예측 결과를 반환합니다.
#       fit() 메서드는 이러한 학습 과정에서 하이퍼파라미터를 설정하거나, 모델이 어떻게 학습되었는지에 대한 정보를 반환하기도 합니다. 
#       모델이 학습될 때, 데이터가 어떻게 처리되는지를 알 수 있는 다양한 정보(예를 들어, 각 특성의 중요도)를 제공하기도 합니다.
# 셋째 줄: 학습된 랜덤 포레스트 모델을 이용하여 X_test 데이터에 대한 예측을 수행하고, 예측 결과를 Y_pred 변수에 저장합니다.
#       테스트 데이터인 X_test를 예측한 결과를 Y_pred에 저장합니다. predict() 메서드는 모델이 학습된 후, 새로운 데이터를 입력받아 예측 결과를 반환합니다.
# 넷째 줄: 학습된 랜덤 포레스트 모델의 성능을 측정합니다. score() 메서드는 
#          모델의 정확도(accuracy)를 계산합니다. X_train, Y_train 데이터를 이용하여 학습한 모델에 대한 정확도를 계산합니다.
#          랜덤 포레스트 모델을 사용하여 ' 학습 데이터 '인 X_train과 그에 해당하는 ' 정답 데이터 '인 Y_train에 대한 정확도(accuracy)를 계산합니다. 
#          score() 메서드는 입력된 데이터셋과 그에 해당하는 정답 데이터셋을 모델에 적용하여, 예측 결과와 정답 데이터를 비교한 후, 그 정확도를 계산합니다. 
#          랜덤 포레스트(Random Forest)는 여러 개의 결정 트리(Decision Tree)를 결합하여 
#          학습하는 알고리즘으로, 각각의 결정 트리가 독립적으로 예측을 수행하고, 
#          결과를 결합하여 최종 예측 결과를 도출합니다. 이는 과적합(overfitting)을 방지하고, 성능을 높일 수 있는 장점이 있습니다.

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)

submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('titanic.csv', index=False)

svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
svc.score(X_train, Y_train)

# Python의 scikit-learn 라이브러리를 사용하여 서포트 벡터 머신
#         (Support Vector Machine, SVM) 모델을 학습하고, 테스트 데이터에 대한 예측을 수행하는 과정입니다.
# 첫째 줄: SVC()로 설정하여 SVM 모델을 생성합니다.
# 둘째 줄: X_train, Y_train 데이터를 이용하여 SVM 모델을 학습합니다.
# 셋째 줄: 학습된 SVM 모델을 이용하여 X_test 데이터에 대한 예측을 수행하고, 예측 결과를 Y_pred 변수에 저장합니다.
# 넷째 줄: 학습된 SVM 모델의 성능을 측정합니다. score() 메서드는 모델의 정확도(accuracy)를 계산합니다. 
#           X_train, Y_train 데이터를 이용하여 학습한 모델에 대한 정확도를 계산합니다.
# 서포트 벡터 머신(SVM)은 결정 경계(decision boundary)와 각 클래스에 속한 데이터 샘플과의 거리(margin)를 
# 최대화하는 초평면(hyperplane)을 찾아서 분류하는 알고리즘입니다. 
# 이를 통해 데이터셋의 차원이 높아지더라도 일정한 정확도를 유지할 수 있습니다. 
# SVM은 선형 분류 뿐만 아니라 커널 기법(kernel trick)을 이용하여 비선형 결정 경계를 찾을 수 있습니다.
# SVM에 대한 자세한 설명은 다음과 같습니다.
# scikit-learn 라이브러리는 파이썬에서 머신 러닝을 구현하는 데 유용한 라이브러리 중 하나입니다. 이 라이브러리에서 제공하는 서포트 벡터 머신(Support Vector Machine, SVM) 모델은 분류(Classification)와 회귀(Regression) 문제를 해결하는 데에 사용됩니다.
# SVM 모델은 데이터를 분류하는 결정 경계(Decision Boundary)를 찾는 알고리즘입니다. SVM 모델은 다른 분류 알고리즘들과는 달리, 마진(Margin)을 최대화하는 결정 경계를 찾는 방식으로 동작합니다. 마진이란, 결정 경계와 가장 가까운 데이터 샘플과의 거리를 의미합니다.
# SVM 모델은 또한 커널(Kernel) 함수를 사용하여 비선형 결정 경계를 찾을 수 있습니다. 커널 함수는 데이터를 고차원 공간으로 변환하여 결정 경계를 찾는 방식입니다. scikit-learn 라이브러리에서는 선형, 다항식, 레이디얼 베이스 함수(Radial Basis Function, RBF) 등 다양한 커널 함수를 제공합니다.
# SVM 모델을 scikit-learn에서 구현할 때, 다음과 같은 단계를 따릅니다.
# 데이터 준비: SVM 모델을 학습시키기 위해서는 데이터셋을 준비해야 합니다. 이때, 입력 데이터와 출력 데이터를 분리하고, 데이터를 정규화하는 등의 전처리 과정을 수행할 수 있습니다.
# 모델 생성: SVM 모델을 생성하고, 모델의 하이퍼파라미터를 설정합니다. SVM 모델에서 주요한 하이퍼파라미터는 커널 타입, C 값, gamma 값 등이 있습니다. 이 값들은 모델 성능에 큰 영향을 미칩니다.
# 모델 학습: 생성한 SVM 모델에 학습 데이터셋을 적용하여 모델을 학습시킵니다. 이때, fit() 함수를 사용하여 모델을 학습시킬 수 있습니다.
# 모델 예측: 학습된 SVM 모델을 이용하여 새로운 데이터를 예측합니다. 
# 이때, predict() 함수를 사용하여 예측 값을 출력할 수 있습니다.
# SVM 모델은 분류 문제뿐만 아니라 회귀 문제도 해결할 수 있습니다. 
# SVM은 지도 학습 알고리즘 중 하나로, 분류(classification) 및 회귀(regression) 문제에 모두 사용할 수 있습니다. SVM은 데이터 포인트 간의 경계(margin)를 최대화하는 결정 경계(decision boundary)를 찾는 것을 목적으로 합니다. SVM은 이상치(outliers)에 강하고 선형 및 비선형 분류에 모두 적용할 수 있으며, 고차원 데이터에서 잘 동작합니다.

# scikit-learn에서 SVM을 사용하기 위해서는 다음과 같은 단계를 거칩니다.

# 데이터셋을 로드합니다.
# 데이터셋을 분할하여 학습(train)과 검증(validation) 데이터셋으로 나눕니다.
# SVM 모델을 정의합니다. 이때 모델의 하이퍼파라미터(hyperparameters)를 설정합니다. 예를 들어, SVM에서는 kernel, C, gamma 등의 하이퍼파라미터를 설정할 수 있습니다.
# SVM 모델을 학습합니다. 학습 데이터셋을 사용하여 SVM 모델을 학습합니다.
# 검증 데이터셋을 사용하여 SVM 모델을 평가합니다. 검증 데이터셋을 사용하여 SVM 모델의 성능을 평가합니다.
# 하이퍼파라미터를 조정합니다. SVM 모델의 하이퍼파라미터를 조정하여 모델의 성능을 개선할 수 있습니다.
# 모델을 새로운 데이터에 적용합니다. 학습된 SVM 모델을 사용하여 새로운 데이터에 대해 예측(prediction)할 수 있습니다.
# 다음은 scikit-learn에서 SVM 모델을 구현하는 예제 코드입니다.
# python
# Copy code
# from sklearn import datasets
# from sklearn.model_selection import train_test_split
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score
# # 데이터셋 로드
# iris = datasets.load_iris()
# X = iris.data
# y = iris.target
# # 데이터셋 분할
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# # SVM 모델 정의
# svm_model = SVC(kernel='linear', C=1, gamma=0.1)
# # SVM 모델 학습
# svm_model.fit(X_train, y_train)
# # SVM 모델 검증
# y_pred = svm_model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print('Accuracy:', accuracy)

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)

submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('titanic.csv', index=False)

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
knn.score(X_train, Y_train)
# 위 코드는 scikit-learn 라이브러리의 KNeighborsClassifier 모델을 사용하여 
# k-최근접 이웃(K-Nearest Neighbors, KNN) 알고리즘을 구현하는 예제입니다. 
# KNN 알고리즘은 지도 학습 알고리즘 중 하나로, 분류(classification) 및 회귀(regression) 문제에 모두 사용할 수 있습니다.
# 위 코드에서 KNeighborsClassifier 모델을 생성하고, n_neighbors 매개변수를 3으로 
# 설정하여 3-최근접 이웃 알고리즘을 사용합니다. 이 알고리즘은 새로운 데이터 
# 포인트에 대해 가장 가까운 3개의 이웃 데이터 포인트를 선택하고, 그 이웃들의 클래스를 
# 보고 가장 많은 클래스를 예측 클래스로 선택합니다.
# 이후 fit 메서드를 사용하여 KNN 모델을 학습시킵니다. 이때 X_train은 학습 데이터셋의 
# 특성(features)을, Y_train은 학습 데이터셋의 레이블(labels)을 나타냅니다.
# predict 메서드를 사용하여 X_test 데이터셋의 예측 레이블을 계산합니다. 
# 이때 Y_pred 변수에 예측 결과가 저장됩니다.
# 마지막으로 score 메서드를 사용하여 학습된 KNN 모델의 정확도(accuracy)를 계산합니다. 
# 이때 X_train과 Y_train을 사용하여 학습된 모델을 평가합니다.
# 즉, 위 코드는 KNN 알고리즘을 사용하여 분류 문제를 해결하는 예제 코드입니다.

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)

submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('titanic.csv', index=False)